In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from pathlib import Path
from typing import cast

import keras
import tensorflow as tf

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=12 * 1024)],
)
logical_gpus = tf.config.list_logical_devices("GPU")
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

In [ ]:
RANDOM_SEED = 709

BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data/cat-and-dog"
TRAIN_DIR = DATA_DIR / "training_set/training_set"
TEST_DIR = DATA_DIR / "test_set/test_set"
CATEGORIES = ["cats", "dogs"]
img_w, img_h, img_ch = IMAGE_TARGET_SIZE = (224, 224, 3)
BATCH_SIZE = 128
EPOCHS = int(1e6)

## Load data

In [ ]:
train_ds = cast(
    tf.data.Dataset,
    keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        validation_split=0.2,
        subset="training",
        seed=RANDOM_SEED,
        image_size=(img_h, img_w),
        batch_size=BATCH_SIZE,
    ),
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
val_ds = cast(
    tf.data.Dataset,
    keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        validation_split=0.2,
        subset="validation",
        seed=RANDOM_SEED,
        image_size=(img_h, img_w),
        batch_size=BATCH_SIZE,
    ),
)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
total_train = len(train_ds) * BATCH_SIZE
total_val = len(val_ds) * BATCH_SIZE

total_train, total_val

## Model definition

In [ ]:
model = keras.Sequential(
    [
        keras.layers.Input(IMAGE_TARGET_SIZE),
        keras.layers.Rescaling(1 / 255),
        keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"),
        keras.layers.MaxPooling2D(pool_size=3, strides=2),
        keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu"),
        keras.layers.MaxPooling2D(pool_size=3, strides=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu"),
        keras.layers.MaxPooling2D(pool_size=3, strides=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu"),
        keras.layers.MaxPooling2D(pool_size=3, strides=2),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="catdog",
)

model.summary()

In [ ]:
# tensorboard = keras.callbacks.TensorBoard(
#     str(BASE_DIR / ".tensorboard"),
#     histogram_freq=5,
#     write_images=True,
# )

sgd = keras.optimizers.SGD(
    learning_rate=1e-2,
    momentum=0.9,
)
lr_red = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=1 / 10,
    mode="min",
    min_lr=1e-4,
    verbose=True,
)
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=int(1e-4),
    patience=5,
    mode="min",
    start_from_epoch=10,
)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=str(DATA_DIR / "catdog.keras"),
    monitor="val_loss",
    verbose=True,
    save_best_only=True,
    mode="min",
)

model.compile(
    loss="binary_crossentropy",
    optimizer=sgd,
    metrics=["accuracy", "precision", "recall"],
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    steps_per_epoch=total_train // BATCH_SIZE,
    validation_steps=total_val // BATCH_SIZE,
    callbacks=[lr_red, early_stop, checkpoint],
)
results = history.history
results

In [ ]:
test_ds = cast(
    tf.data.Dataset,
    keras.utils.image_dataset_from_directory(
        TEST_DIR,
        seed=RANDOM_SEED,
        image_size=(img_h, img_w),
        batch_size=BATCH_SIZE,
        shuffle=False,
    ),
)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)
total_test = len(test_ds) * BATCH_SIZE
total_test

In [ ]:
evals = model.evaluate(
    test_ds,
    verbose=1,
    steps=total_test // BATCH_SIZE,
)